# Low light Image enhancement 
In this notebook, we illustrate the use of the Zero-dce model for low light image enhancement using custom postprocessor with DeGirum PySDK

### Define custom postprocessor

In [ ]:
from __future__ import annotations

from typing import Any

import cv2
import numpy as np
from degirum.postprocessor import InferenceResults, log_wrap
from degirum._model_param_helpers import model_shape_get


class EnhancementPostprocessor(InferenceResults):
    """Convert a flattened RGB float‑tensor ``(1, H×W, 3)`` produced by
    a low‑light / enhancement model into a display‑ready frame.

    Public attributes / properties
    ------------------------------
    * **image** – the original input frame (unchanged)
    * **output_image** – enhanced frame, *uint8 BGR*, resized to match the input
    * **image_overlay** – simple alias to :pyattr:`output_image` so built‑in
      SDK viewers show the *after* frame automatically.
    """

    @staticmethod
    def _float_to_uint8(img_f32: np.ndarray) -> np.ndarray:
        """Return a uint8 RGB image suitable for Pillow / OpenCV display.

        * If values lie in **[0 … 1]** → multiply by 255.
        * If values lie in **[0 … 255]** → just round & cast.
        * Otherwise (rare) → scale by max value to preserve contrast.
        """
        lo, hi = img_f32.min(), img_f32.max()
        if hi <= 1.01:                   
            img_u8 = img_f32 * 255.0
        elif hi <= 255.0:               
            img_u8 = img_f32
        else:                           
            img_u8 = img_f32 / hi * 255.0
        return img_u8.round().astype(np.uint8)

    @log_wrap
    def __init__(self, *args: Any, **kwargs: Any):
        super().__init__(*args, **kwargs)

        try:
            raw_tensor = np.asarray(self._inference_results[0]["data"],
                                     dtype=np.float32)
        except Exception as e:
            raise RuntimeError(
                "EnhancementPostprocessor expects inference_results[0]['data'] "
                "to contain the flat output tensor.  Didn't find it: " + str(e)
            ) from e

        raw_tensor = raw_tensor.squeeze(0)  # (H×W, 3)

        try:
            _, H, W, _ = model_shape_get(self._model_params, 0, 4)
        except Exception:
            H, W = 400, 600  # fallback for Zero‑DCE reference

        if raw_tensor.shape[0] != H * W:
            raise ValueError(
                f"Flat tensor length {raw_tensor.shape[0]} does not match H×W "
                f"({H}×{W}={H*W}).  Check model output/shape assumptions."
            )

        img_rgb_f32 = raw_tensor.reshape(H, W, 3)
        img_rgb_u8 = self._float_to_uint8(img_rgb_f32)
        img_bgr_u8 = cv2.cvtColor(img_rgb_u8, cv2.COLOR_RGB2BGR)

        if hasattr(self._input_image, "shape"):
            orig_h, orig_w = self._input_image.shape[:2]
        else:  # Pillow Image
            orig_w, orig_h = self._input_image.size

        self._output_image: np.ndarray = cv2.resize(
            img_bgr_u8, (orig_w, orig_h), interpolation=cv2.INTER_LINEAR
        )

    @property
    def output_image(self) -> np.ndarray:
        """Enhanced frame – *uint8 BGR*, same resolution as the input."""
        return self._output_image


    @property
    def image_overlay(self):  
        return self._output_image


In [ ]:
import degirum as dg, degirum_tools
import cv2

inference_host_address = "@local"
zoo_url = 'degirum/hailo'
token=''
device_type='HAILORT/HAILO8L'
model_name = 'zero_dce--400x600_quant_hailort_hailo8l_1'

model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type,
    custom_postprocessor= EnhancementPostprocessor,
)



# Load an image
enhanced_result = model('../assets/dark_room.jpg')


with degirum_tools.Display() as display:
    display.show_image(enhanced_result.output_image)